In [181]:
from collections import defaultdict
import pandas as pd
from urllib.parse import unquote
import tqdm
import os
from bs4 import BeautifulSoup  
import numpy as np
import urllib


In [182]:
path = 'raw\\raw\\'

In [183]:
import re

In [184]:
re.findall(r'/wiki/+\w*,?\b','/wiki/fsd_de_ded ddfdfsdsd sds ds sdsd /wiki/dfdf')

['/wiki/fsd_de_ded', '/wiki/dfdf']

In [185]:
topics = {}
for i, file in enumerate(os.listdir(path)):
    topics[file[:-5]] = i

In [186]:
len(topics)

2381

In [187]:
matrix = np.zeros((len(filenames), len(filenames)))

In [188]:
matrix.shape

(2380, 2380)

In [157]:
df = defaultdict(lambda: [])
for i in tqdm.tqdm_notebook(os.listdir(path)):
    with open (path+'\\'+i, encoding="UTF-8") as file:
        read = unquote(file.read())
        cur_topic = i[:-5]
        link_site = re.findall(r'/wiki/.+?\b',read)
        for link in link_site:
            if link in topics.keys():
                df[i.split('.')[0]].append(link.split('/')[-1])
                matrix[topics[cur_topic]][topics[link[6:]]] = 1
                
        

In [158]:
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [189]:
for file in tqdm.tqdm_notebook(filenames):
    with open(f"raw/raw/{file}", encoding='utf-8') as f:
        cur_topic = file[:-5]
        bs = BeautifulSoup(f.read(), "html.parser")
        all_a = bs.find_all("a")
        for link in all_a:
            if not 'href' in link.attrs:
                continue
            href = urllib.parse.unquote(link['href'])
            if href.startswith("/wiki/") and href[6:] in topics:
                matrix[topics[cur_topic]][topics[href[6:]]] = 1

            

IndexError: index 2380 is out of bounds for axis 0 with size 2380

In [160]:
for i in matrix:
    for j in range(len(i)):
        if i[j] == 0:
            i[j] = 0.001

In [161]:
row_sums = matrix.sum(axis=1)
norm_matrix = matrix / row_sums[:, np.newaxis]
vector = np.array([0.2]*len(filenames))

In [162]:
for _ in range(100):
    vector = norm_matrix.transpose().dot(vector)

In [163]:
np.take(vector, [1, 2])

array([0.2, 0.2])

In [164]:
rank = {}
for i, value in enumerate(vector):
    rank[i] = value

In [165]:
def page_rank(rank, n=10, word=None):
    if word is not None:
        new_topics = [ind for topic, ind in topics.items() if word.lower() in topic.lower()]
    if word is not None:
        new_rank = {}
        for ind in new_topics:
            new_rank[ind] = rank[ind]
    else:
        new_rank = rank.copy()
    items = sorted(new_rank.items(), key=lambda t: t[1])[::-1][:n]
    for index, value in items:
        print(f"{filenames[index][:-5]}: {value}")

In [166]:
page_rank(rank=rank)

Ярл: 0.20000000000002569
Япония: 0.20000000000002569
Яков_V_(король_Шотландии): 0.20000000000002569
Яков_VI_(король_Шотландии): 0.20000000000002569
Яков_I_(король_Англии): 0.20000000000002569
Яков_II_(король_Англии): 0.20000000000002569
Яичко: 0.20000000000002569
Языки_мира: 0.20000000000002569
Я_—_легенда_(фильм): 0.20000000000002569
Юты: 0.20000000000002569


In [167]:
page_rank(rank=rank, word='армстронг')

Пэт_Гэрретт_и_Билли_Кид: 0.20000000000002569
Клан_Скотт: 0.20000000000002569
Билли_Джо_Армстронг: 0.20000000000002569
Армстронг_(фамилия): 0.20000000000002569
Армстронг_(округ,_Техас): 0.20000000000002569
Армстронг_(округ,_Пенсильвания): 0.20000000000002569
Армстронг_(округ): 0.20000000000002569
Армстронг_(лунный_кратер): 0.20000000000002569
Армстронг: 0.20000000000002569
Армстронг-Уитворт: 0.20000000000002569
